# Maximum Likelihood
At the end of the previous section, we discussed the concept of estimating the parameters of the simple regression model. In this section, we will explore a method of estimation known as *maximum likelihood estimation* (MLE) that is applicable to both simple linear models, as well as many of the more complex models we will see later on the course.

## MLE vs OLS
At the beginning of this lesson, we demonstrated how the simple regression line could be fit by minimising the sum of squared errors. Effectively, we were finding the line that *minimised* the variance of the data around the regression line. This is known as the method of *ordinary least squares* (OLS). OLS is useful because it is very simple to conceptualise and results in a simple set of equations for finding estimates. Because of this, OLS is generally considered *the* method used for estimating normal linear models. However, the main problem with OLS is that it is largely only applicable to these types of models. For instance, models that do not assume a normal distribution or those that assume more complex correlational and variance structures cannot be estimated with OLS. As such, OLS is actually a very niche approach that gets abandoned very quickly as soon as models get more complex. Because of this, it is much more helpful in terms of thinking *generally* to consider a *likelihood* approach to estimation[^searlefoot]. This is widely applicable across may different types of model, as well as being fundamental to Bayesian approaches to statistics.

## How Does MLE Work?
Fundamentally, MLE is based on finding parameter values that make a certain value, known as the *likelihood*, as *big* as possible. The value of likelihood is calculated using the *likelihood function*, denoted 

$$
L\left(\boldsymbol{\theta}|\mathcal{D}\right),
$$

where $\mathcal{D}$ represents the data we have collected and $\boldsymbol{\theta}$ is a generic representation of any set of parameters. For instance, in simple regression, $\boldsymbol{\theta} = \{\beta_{0},\beta_{1},\sigma^{2}\}$. In order to understand this, we can make an equivalence between the likelihood and probability. When placed in probabilitstic terms, we have

$$
L\left(\boldsymbol{\theta}|\mathcal{D}\right) = P\left(\mathcal{D}|\boldsymbol{\theta}\right).
$$

 In words, this is saying that the likelihood of a set of paramater values, given some data, is the *same* as evaluating the probability of the data, assuming those parameter values are true. The key point here is that MLE is based on evaluating the *probability of the data*, given some values of the parameters[^bayesfoot]. We can think about it as taking a guess for the parameter values, then calculating how probable those values make the data we have collected. It is like asking the question: "how likely would it have been to collect the data that we have collected, if these were the parameter values?". By searching through lots of different possible combinations of parameter values, the aim is to find the specific combination that leads to the *highest probability* of the data. In other words, the values that *maxmimise the likelihood*.

### The Likelihood Function
So how do we evaluate the likelihood function? In brief, if we want the likelihood of our entire dataset, we need to *multiply* the probability of each data value, using some guesses for the parameter values. We can then repeat this for some other guesses. If the likelihood gets *larger* then our new guesses make the data more probable than our old guesses. We will see shortly how to do this in a more principled way than just guessing, but this should give you enough of an idea at this point.

So how do we calculate the probabilities of each data point? Remember that our core assumption for simple regression is that

$$
y_{i} \sim \mathcal{N}\left(\beta_{0} + \beta_{1}x_{i}, \sigma^{2}\right).
$$

The probability of any value from a normal distribution can be calculated using its *probability density function*, which is given by

$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x - \mu)^{2}}{2\sigma^{2}}}.
$$

This can look a little complicated, but notice that the only values we need to plug-in here are $\mu$, $\sigma^{2}$ and some value for $x$. For instance, if we wanted to know the probability of a value of 10 from a normal distribution with a mean of 9 and a variance of 1, then we have

In [35]:
x      <- 10
mu     <- 9
sigma2 <- 1

P.x <- 1/sqrt(2*pi*sigma2) * exp(-((x-mu)^2)/(2*sigma2))
print(P.x)

[1] 0.2419707


We can then reframe the expression for the probability density using our model to give

$$
P(y_{i}) = \frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(y_{i} - (\beta_{0} + \beta_{1}x_{i}))^{2}}{2\sigma^{2}}}.
$$

So, if we insert some guesses for $\beta_{0}$, $\beta_{1}$ and $\sigma^{2}$, we can calculate the probability of any of our data values. This effectively tells us the probability of that datapoint, assuming our guesses are correct. The likelihood function then involves multiplying all these values together to give

$$
L(\beta_{0},\beta_{1},\sigma^{2}|\mathbf{y}) = P\left(y_{1}\right) \times P(y_{2}) \times \dots \times P(y_{n})
$$

which we can more compactly express using Big Pi notation (see the box below)

$$
\begin{align*}
    L(\beta_{0},\beta_{1},\sigma^{2}|\mathbf{y}) &= \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(y_{i} - (\beta_{0} + \beta_{1}x_{i}))^{2}}{2\sigma^{2}}} \\
    &= \prod_{i=1}^{n} P(y_{i})
\end{align*}
$$

`````{admonition} Big Pi Notation
:class: tip
Big pi notation, denoted by the captitol Greek letter $\Pi$, is used as a shorthand for *multiplication*. Below the big Pi, we indicate the notation for our index across these multiplications, as well as our starting index value. Above the big Pi, we indicate the value where we stop. So the notation

$$
P = \prod_{i=1}^{3} y_{i}
$$

is equivalent to

$$
P = y_{1} \times y_{2} \times y_{3} ,
$$

So, the notation says that our index is called $i$ and that we start it at $1$. We then keep going until $i = 3$, at which point we stop. 

This has a direct connection to a for-loop. So, in code, this is the same as shortening

```R
P <- y[1] * y[2] * y[3]
```

to

```R
P <- 1
for (i in 1:3){
    P <- P * y[i]
}
```
So, you can think of the Big Pi notation as a *multiplication loop* over a certain set of indices.
`````

### A Concrete Example in `R`
To get a more concrete sense of calculating the likelihood, let us use the `mtcars` data again. Furthermore, let us say that we have guessed that $\beta_{0} = 30$, $\beta_{1} = -5$ and $\sigma^{2} = 1$. If we are assuming that the data have come from a normal distribution, we can calculate the probability of the first value of `mpg` using

In [1]:
beta.0 <- 30
beta.1 <- -5
sigma2 <- 1
mu     <- beta.0 + beta.1*mtcars$wt[1]
P.y1   <- dnorm(mtcars$mpg[1], mean=mu, sd=sqrt(sigma2))
print(P.y1 )

[1] 8.926166e-05


where the function `dnorm` returns the *density* of the normal distribution for the given data (so we do not need to calculate this manually, as we did earlier). The probability of the second value of `mpg` would then be

In [3]:
mu  <- beta.0 + beta.1*mtcars$wt[2]
P.y2 <- dnorm(mtcars$mpg[2], mean=mu, sd=sqrt(sigma2))
print(P.y2)

[1] 2.125155e-07


and so on. The find out the overall likelihood for the *whole* dataset, we just need to *multiply* these probabilities. However, this can cause computational problems[^likprobsfoot], so it is more usual to sum the *log* of these probabilities. This gives the *log-likelihood*.

In [4]:
mu     <- beta.0 + beta.1*mtcars$wt
loglik <- sum(dnorm(mtcars$mpg, mean=mu, sd=sqrt(sigma2), log=TRUE))
print(loglik)

[1] -780.7884


This value is not particularly interpretable, but this does not matter. All we want to do is make it as *big* as possible. In the context of a negative value, this means our aim is to make the likelihood as *positive* as possible. In principle, we just need to keep trying guesses for the parameters to see which ones make the log-likelihood as large as possible. Once we cannot make it any bigger, the estimation is complete.

### Exact Solutions
An obvious issue with the scheme above is that searching through many combinations of guesses for the parameters is not particularly efficient or practical. In fact, there is an infinite number of values we could choose, as well as an infinite number of combinations. So how can we possibly find the values we need? In order to do so, there are two options. For some simple problems, the equations that maximise the likelihood have already been worked out using the tools of calculus. As such, the equation for the likelihood can be solved to find those values that guarantee a maximum. Normal linear models are one such example. By assuming a Gaussian distribution for the outcome variables, the ML estimates for a simple regression model are

$$
\begin{align*}
    \hat{\beta}_{1} &= \frac{\sum{\left(x_{i} - \bar{x}\right)\left(y_{i} - \bar{y}\right)}}{\sum{\left(x_{i} - \bar{x}\right)^{2}}}\\
    \hat{\beta}_{0} &= \bar{y} - \hat{\beta}_{1}\bar{x}, \\ 
\end{align*}
$$

which agree with what we saw earlier for OLS. As such, both MLE and OLS agree on the values of the intercept and slope. Because of this, we can conceptualise estimation of linear models either in terms of least-squares *or* in terms of the likelihood as, practically, the outcome is the same. Unfortunately, for more complex models, there are no exact solutions for maximising the likelihood. In these cases, we have to turn to computational methods in the form of *iterative* MLE.

### Iterative MLE
For situations where it is not possible to find an exact solution, we rely on computer algorithms to search through many possible combinations of values to find which one *maximises* the log-likelihood. These are known more generally as *optimisation* algorithms and are a complex topic in numerical computing. For our purpose, we do not really need to understand how these algorithms work. All we really need to know is that they use rules and heuristics to explore the space of all possible parameter values in order to find values that the algorithm thinks make the log-likelihood the largest.

Within `R` the generic functions `optim()`, `nlm()` and `nlminb()` can all be used to do this. In the example below, we choose `nlm()` (*nonlinear minimisation*) for its general robustness for MLE problems. This function needs some starting guesses for the parameters, so in the example below we set $\hat{\beta}_{0} = \bar{y}$, $\hat{\beta}_{1} = 0$ and $\sigma = \text{SD}(y)$. As the name implies, this function *minimises*, so we return the *negative* log-likelihood instead[^minmaxfoot]. After running the iterative ML estimation, we get 

In [29]:
set.seed(123)
x <- mtcars$wt
y <- mtcars$mpg

# Define negative log-likelihood
neg_loglik <- function(params) {
  beta.0 <- params[1]
  beta.1 <- params[2]
  sigma  <- params[3]

  if (sigma <= 0) return(1e10)

  mu     <- beta.0 + beta.1*x    
  loglik <- sum(dnorm(y, mean=mu, sd=sigma, log=TRUE)) # log-likelihood
  return(-loglik)                                      # -ve log-likelihood
}

# Starting values (guesses for intercept, slope and SD)
init_params <- c(mean(y), 0, sd(y))

# Run optimisation
mle <- nlm(f=neg_loglik, p=init_params)

# Print results
mle_pars <- mle$estimate[1:2]
names(mle_pars) <- c("beta.0", "beta.1")
print(mle_pars)


   beta.0    beta.1 
37.285127 -5.344472 


Which we can compare to the OLS results `R` gives us when using the `lm()` function

In [7]:
print(coef(lm(mpg ~ wt, data=mtcars)))

(Intercept)          wt 
  37.285126   -5.344472 


As we can see, these are very close, showing how iterative MLE can be applied in many cases, even those where exact solutions exist. Although we do not need this for simple linear models, we will see later on the course how iterative MLE is necessary for Generalised Linear Models, Linear Mixed Models and Generalised Linear Mixed Models.

### Restricted Maximum Likelihood
In the examples above, your may have noticed that we neglected to show the estimates for $\sigma^{2}$. This was not an accident. In terms of exact solutions, the variance estimates produced by OLS and MLE are

$$
\begin{align*}
    \hat{\sigma}^{2}_{\text{(OLS)}} &= \frac{1}{n - k} \sum_{i=1}^{n} e_{i}^{2} \\
    \hat{\sigma}^{2}_{\text{(ML)}}  &= \frac{1}{n} \sum_{i=1}^{n} e_{i}^{2} \\
\end{align*}
$$

So, the OLS variance estimate implements a generalisation of Bessel's correction, by dividing the sum of squared errors by $n - k$ (where $k$ is the number of parameters). The ML estimate, by comparison, simply divides by $n$. Unless we have the entire population at our disposal, this will give a *biased* estimate for the variance, effectively *underestimating* it compared to the true value. We can see this bias in the results from iterative MLE, where the estimate was

In [39]:
print(mle$estimate[3])

[1] 2.949163


compared to the OLS estimate of

In [40]:
print(summary(lm(mpg ~ wt, data=mtcars))$sigma)

[1] 3.045882


The reason this happens is because MLE estimates the variance without taking into account that $\hat{\beta}_{0}$ and $\hat{\beta}_{1}$ are *estimates*. MLE treats these values as *known constants* and calculates the variance accordingly. As such, MLE is not *wrong* in its calculations, but it is wrong in its assumptions when estimating the variance. If these values were the true population values, then MLE would be correct. However, they are not.

In order to fix this, a variation of MLE was developed specifically for estimating variances known as *restricted* maximum likelihood (REML). The way this works is a bit complicated and somewhat beyond the scope of this lesson. In effect, REML estimates the variance by first *removing* the effects of the predictors from the data. The resultant errors are then used to estimate the variance by running standard MLE (hence why it is sometimes known as *residual* maximum likelihood). The removal of the effects allows the correction to be automatically taken into account, meaning the variance estimate is *unbiased*. How this removal is achived gets quite complicated, but this should give you enough of a flavour of how REML works. 



`````{admonition} Degrees of Freedom
:class: tip
The value $n-k$ in the denominator of the unbiased variance estimate is known as the *residual degrees of freedom*. The concept of degrees of freedom is something of a difficult idea, particularly because the best way to understand them is to take a geometric perpsective that requires a much deeper understanding of linear models. Without this perspective, we have to rely on slightly vague descriptions such as "the number of independent values after estimation". ...
`````

[^densityfoot]: This is the area under the normal curve, equivalent to the *probability* of a specific value.

[^likprobsfoot]: This is often a problem of just getting values of 0 due to issues with computational precision when working with many small probabilities. Taking logs not only changes the scale so that this does not happen, but it also turns *multiplication* into *summation*. Historically, this made calculating the likelihood much easier by hand. If you ever want to get back to the likelihood value, you can just undo the logs by using `exp(loglik)`.

[^bayesfoot]: This may seem like the wrong quantity. Surely, we are interested in $P(\boldsymbol{\theta}|\mathcal{D})$? In other words, finding the parameters that are most probable, given the data. Unfortunately, evaluating the probability $P(\boldsymbol{\theta}|\mathcal{D})$ requires Bayesian methods and so cannot be evaluated from a purely Frequentist perspective. This is why the likelihood has such an awkward definition.

[^minmaxfoot]: Making the *negative* log-likelihood *smaller* is the same as making the *positive* log-likelihood *bigger*. All we need to do to turn a maxmisation problem into a minimisation problem is to multiply the value returned by the function by $-1$.

[^searlefoot]: This is the perspective taken by [McCulloch, Searle & Neuhaus (2008)](https://www.librarysearch.manchester.ac.uk/permalink/44MAN_INST/1r887gn/alma9930787964401631), who are leading experts on the use of linear models and their derivatives within statistics. This book gives everything you need to understand about the mathematical theory behind this framework, though it is not for the faint of heart!